In [1]:
import dlc_bci as bci
import torch
from torch.autograd import Variable
from torch import nn
from torch import optim
from torch import Tensor
from torch.nn import functional as F
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

## Load the data

In [141]:
train_input , train_target = bci.load ( root = "./ data_bci")
test_input , test_target = bci.load ( root = "./ data_bci " , train = False )

train_target = Variable(train_target)
train_input = Variable(train_input.unsqueeze(dim=2))
test_input = Variable(test_input.unsqueeze(dim=2))
test_target = Variable(test_target)

mu, std = train_input.data.mean(), train_input.data.std() 
train_input.data.sub_(mu).div_(std)
test_input.data.sub_(mu).div_(std)


print('train_input size:' , train_input.size())
print ( 'train_target size:' , train_target.size())
print ('test_input size:' , test_input.size())
print ('test_target size:' , test_target.size())

train_input size: torch.Size([316, 28, 1, 50])
train_target size: torch.Size([316])
test_input size: torch.Size([100, 28, 1, 50])
test_target size: torch.Size([100])


In [142]:
# The function computes the accuracy
def accuracy(output,target):
    return (output.float()==target.float()).float().sum()/len(output)

## Convolutional neural network

In [143]:
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.conv1 = nn.Conv2d (28, 32, kernel_size = (1,5))
        self.avg_pool1 = nn.AvgPool2d(kernel_size = (1,4))
        self.conv2 = nn.Conv2d (32 , 64, kernel_size = (1,5))
        self.fc1 = nn.Linear (448 , 200)
        self.fc2 = nn.Linear (200 , 2)
        
    def forward (self , x):
        x = self.conv1(x)
        x = F.relu(self.avg_pool1(x))
        x = F.relu(self.conv2(x))
        x = x.view (-1, 448)
        x = F.relu ( self.fc1 (x))
        x = self.fc2(x)
                   #x = F.relu(F.max_pool2d(self.conv1(x), kernel_size = 3, stride =3) )
        #x = F.relu (F.max_pool2d ( self.conv2(x), kernel_size =2, stride =2) )
        return x

In [156]:
net = Net()

In [157]:
### train the model
epoch = 1000
batch_size = 20
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(),lr = 0.001,momentum = 0.5)
i = 0
for e in range(epoch):
    for b in range(0,train_input.size(0),batch_size):
        input_ = train_input.narrow(0,b,min(batch_size,train_input.size(0)-b))
        target = train_target.narrow(0,b,min(batch_size,train_input.size(0)-b))
        output = net(input_).view(min(batch_size,train_input.size(0)-b),2)
        loss = criterion(output,target)
        net.zero_grad()
        loss.backward()
        optimizer.step()
    if e%100 == 0: # print the loss every 10 epochs
        output = net(test_input).view(-1,2)
        a,predicted_class = output.max(dim=1)
        output_train = net(train_input).view(-1,2)
        a,predicted_class_train = output.max(dim=1)
        
        #predicted_class = 1*(predicted_class.float().mean(dim=1) > 0.5)
        print(e//100,'loss train:',criterion(output_train,train_target).data[0],'loss test:',criterion(output,test_target).data[0],' accuracy test:',accuracy(predicted_class,test_target).data[0])

0 loss train:  0.6936008930206299 loss test 0.6927070021629333  accuracy test  0.4699999988079071
1 loss train:  0.6863560676574707 loss test 0.6956783533096313  accuracy test  0.4699999988079071
2 loss train:  0.6719890832901001 loss test 0.7013039588928223  accuracy test  0.41999998688697815
3 loss train:  0.6429463624954224 loss test 0.7169969081878662  accuracy test  0.4399999976158142
4 loss train:  0.5761215686798096 loss test 0.6860684156417847  accuracy test  0.5400000214576721
5 loss train:  0.4948302209377289 loss test 0.6379230618476868  accuracy test  0.6399999856948853
6 loss train:  0.4354327619075775 loss test 0.6297763586044312  accuracy test  0.6399999856948853
7 loss train:  0.3722091615200043 loss test 0.6171790957450867  accuracy test  0.6800000071525574
8 loss train:  0.3147490918636322 loss test 0.6212679743766785  accuracy test  0.699999988079071
9 loss train:  0.2581411302089691 loss test 0.6359745860099792  accuracy test  0.7300000190734863


In [98]:
Variable(torch.LongTensor(3).random_(2))

Variable containing:
 1
 0
 0
[torch.LongTensor of size 3]